In [ ]:
import numpy as np
import time
import pandas as pd
import glob
from PIL import Image
from matplotlib import pyplot as plt
import tensorflow as tf

from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score


In [ ]:
tpr_list = []
fpr_list = []
name_list = []
auc_list = []

In [ ]:
def load_one_image(input_path):
    img = Image.open(input_path)
    img = img.resize([size_w,size_h])
    rslt = np.asarray(img)/255
    
    return rslt

def load_many_image(input_path):
    rslt = np.zeros([len(input_path),size_h,size_w,3])
    for i,img_path in enumerate(input_path):
        rslt[i] = load_one_image(img_path)
        
    return rslt

In [ ]:
for data_type in ["ap","lateral"]:
    if data_type=="ap":
        seed = '308'  # your random seed here
    if data_type=="lateral":
        seed = '662'  # your random seed here
    for rnd_seed in ['1','2','3','4','5']:
        data_path = "./DATA/"



        run_name = data_type+"_--"+seed+'_'+rnd_seed

        size_w = 500
        size_h = 600
        tr_ratio = 0.6
        te_ratio = 0.2

        learning_rate = 3e-5
        epoch = 500

        print(run_name)


        # normal =0, abn = 1
        all_path_0 = glob.glob(data_path+"normal/"+data_type+"/*.jpg")
        all_path_1 = glob.glob(data_path+"abnormal/"+data_type+"/*.jpg")
        all_id_0 = [s.split("\\")[-1].split("_")[0] for s in all_path_0]
        all_id_1 = [s.split("\\")[-1].split("_")[0] for s in all_path_1]
        all_unique_id_0 = np.unique(all_id_0)
        all_unique_id_1 = np.unique(all_id_1)
        print(len(all_id_0),"   ", len(all_unique_id_0))
        print(len(all_id_1),"   ", len(all_unique_id_1))


        print(all_path_0[:3])
        print(all_path_1[:3])

        np.random.seed(int(rnd_seed))
        np.random.shuffle(all_path_0)
        np.random.shuffle(all_path_1)

        # normal =0, abn = 1
        path_te_0 = all_path_0[int(len(all_path_0)*(1-te_ratio)):]
        path_te_1 = all_path_1[int(len(all_path_1)*(1-te_ratio)):]

        path_te = np.concatenate([path_te_0,path_te_1])

        target_label_te = np.array( [s.split("/")[2] for s in path_te] )

        target_te = np.asarray(pd.get_dummies(target_label_te))
        # abnormal = [1,0]   normal = [0,1]
        print( np.unique(target_label_te,return_counts=True) )


        input_te = load_many_image(path_te)
        print("reading finished")

        model = tf.keras.models.load_model(run_name+"_best.hdf5")
        print("loading model finished")

        pred_te = model.predict(input_te, batch_size=4)
        auc_te = roc_auc_score(target_te[:,0],pred_te[:,0])
        fpr, tpr, _ = roc_curve(target_te[:,0], pred_te[:,0])

        fpr_list.append(fpr)
        tpr_list.append(tpr)
        name_list.append(run_name)
        auc_list.append(auc_te)

        print(run_name)
        print(auc_te)

In [ ]:
print(pd.DataFrame({'name':name_list,'auc':auc_list}))

In [ ]:
import os
os.makedirs("./ROC_output/",exist_ok=True)

In [ ]:
plt.plot(tpr, tpr)
plt.plot(fpr_list[3], tpr_list[3], marker='.', label="AP_4")
plt.plot(fpr_list[5], tpr_list[5], marker='.', label="Lateral_1")
# axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
# show the legend
plt.legend()
plt.title("ROC curve")
# show the plot
plt.savefig("./ROC_output/ROC_best_2.png",dpi=300)
plt.show()
img = Image.open("./ROC_output/ROC_best_2.png")
img.save("./ROC_output/ROC_best_2.tif",dpi=(300,300))

In [ ]:
plt.plot(tpr, tpr)
plt.plot(fpr_list[0], tpr_list[0], marker='.', label="AP_1")
plt.plot(fpr_list[1], tpr_list[1], marker='.', label="AP_2")
plt.plot(fpr_list[2], tpr_list[2], marker='.', label="AP_3")
plt.plot(fpr_list[3], tpr_list[3], marker='.', label="AP_4")
plt.plot(fpr_list[4], tpr_list[4], marker='.', label="AP_5")
# axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
# show the legend
plt.legend()
plt.title("ROC curve - AP")
# show the plot
plt.savefig("./ROC_output/ROC_AP.png",dpi=300)
plt.show()
img = Image.open("./ROC_output/ROC_AP.png")
img.save("./ROC_output/ROC_AP.tif",dpi=(300,300))

In [ ]:
plt.plot(tpr, tpr)
plt.plot(fpr_list[5], tpr_list[5], marker='.', label="Lateral_1")
plt.plot(fpr_list[6], tpr_list[6], marker='.', label="Lateral_2")
plt.plot(fpr_list[7], tpr_list[7], marker='.', label="Lateral_3")
plt.plot(fpr_list[8], tpr_list[8], marker='.', label="Lateral_4")
plt.plot(fpr_list[9], tpr_list[9], marker='.', label="Lateral_5")
# axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
# show the legend
plt.legend()
plt.title("ROC curve - Lateral")
# show the plot
plt.savefig("./ROC_output/ROC_Lateral.png",dpi=300)
plt.show()
img = Image.open("./ROC_output/ROC_Lateral.png")
img.save("./ROC_output/ROC_Lateral.tif",dpi=(300,300))